<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch syft

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import syft as sy

# Define a simple model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create a federated learning setup
hook = sy.TorchHook(torch)
alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")

# Distribute data among virtual workers
data = torch.rand(32, 784)
target = torch.randint(0, 10, (32,))
federated_data = [(data[:16].send(alice), target[:16].send(alice)),
                  (data[16:].send(bob), target[16:].send(bob))]

# Model and optimizer
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(2):
    model.train()
    for (data, target) in federated_data:
        optimizer.zero_grad()
        output = model(data)
        loss = nn.functional.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}, Loss: {loss.get().item()}")

# Test the model (locally)
model.eval()
test_data = torch.rand(32, 784)
test_target = torch.randint(0, 10, (32,))
output = model(test_data)
test_loss = nn.functional.cross_entropy(output, test_target)
print(f"Test Loss: {test_loss.item()}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import syft as sy

# Define a simple model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize a hook
hook = sy.TorchHook(torch)

# Create virtual workers
alice = sy.VirtualWorker(id="alice", hook=hook)
bob = sy.VirtualWorker(id="bob", hook=hook)

# Distribute data among virtual workers
data = torch.rand(32, 784)
target = torch.randint(0, 10, (32,))
federated_data = [(data[:16].send(alice), target[:16].send(alice)),
                  (data[16:].send(bob), target[16:].send(bob))]

# Model and optimizer
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(2):
    model.train()
    for (data, target) in federated_data:
        optimizer.zero_grad()
        output = model(data)
        loss = nn.functional.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}, Loss: {loss.get().item()}")

# Test the model (locally)
model.eval()
test_data = torch.rand(32, 784)
test_target = torch.randint(0, 10, (32,))
output = model(test_data)
test_loss = nn.functional.cross_entropy(output, test_target)
print(f"Test Loss: {test_loss.item()}")